In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #data visualization
import seaborn as sns
from sklearn.model_selection import train_test_split # Split data to train and test data(after merging in this case)
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix 
import warnings
warnings.filterwarnings('ignore')


# import libraries from tensorflow
from tensorflow import keras
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout,Input, BatchNormalization, LeakyReLU, ReLU
from tensorflow.keras import layers
from tensorflow.keras import optimizers
#from keras.utils.vis_utils import plot_model
from keras.utils import plot_model
#from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.initializers import HeUniform
from tensorflow.keras.regularizers import l2
import joblib

In [3]:
# Test the loaded model
from tensorflow.keras.models import load_model

# Load the entire model
loaded_model = load_model("IL_trained_model_v2.h5")

In [4]:
#Load custom input data
# Import dataset

df_test = pd.read_csv("TestData.csv") #reading the provided dataset
df_test.head(10)

,Orientation Loss,Edge Coverage,Average Thickness,Average Separation,Distance Entropy
0,1037.830566,16.6758,1.8025,2.6209,0.9644
1,342.689941,13.3282,2.1457,3.1926,0.9648
2,762.545898,17.9451,1.6979,2.3449,0.9704
3,465.277466,12.7644,2.3037,3.4747,0.9607
4,984.840271,16.6378,1.8696,2.5217,0.9765
5,125.517296,13.5489,2.4059,2.7382,0.9982
6,1504.066284,19.8257,1.5317,2.2086,0.9603
7,1367.129272,20.7555,1.5467,2.0168,0.9778
8,855.029663,21.8735,1.5065,1.9006,0.9804
9,439.846100,16.5547,1.9643,2.4226,0.9892


In [5]:
# Load the test data scaler from the saved file
loaded_scaler_testdata = joblib.load('scale_testdata.pkl')
# Load the scaler from the saved file ( to get original scale to convert to actual values )
loaded_scaler_originalval = joblib.load('scaler_originalval.pkl')

In [6]:
# fit: calculates the statistics (like mean and standard deviation for StandardScaler)
# from the data that the scaler needs to perform the transformation.
# transform: This step applies the calculated scaling/transformations to the data.
TestData = loaded_scaler_testdata.transform(df_test)
TestData

array([[ 0.3996857 , -0.37723648, -0.2013607 ,  0.39453831, -1.16207511],
       [-0.96035944, -1.25657267,  0.72307299,  1.35493974, -1.13659549],
       [-0.13890975, -0.04382108, -0.48310826, -0.06911534, -0.7798808 ],
       [-0.72051651, -1.40466973,  1.14865727,  1.82884081, -1.3977616 ],
       [ 0.29601   , -0.38721819, -0.02062206,  0.22789178, -0.39131658],
       [-1.38525851, -1.19859994,  1.42394026,  0.59159111,  0.99095285],
       [ 1.31187757,  0.45016854, -0.93077982, -0.29808633, -1.42324122],
       [ 1.04395981,  0.69440524, -0.89037625, -0.62029203, -0.30850781],
       [ 0.04203508,  0.98807768, -0.99865782, -0.81549693, -0.14289028],
       [-0.77027307, -0.40904662,  0.23445915,  0.06141324,  0.41766138]])

In [7]:
# Now we can use the loaded_model to make predictions on your test data
predictions = loaded_model.predict(TestData)

1/1 [==============================] - 0s 51ms/step


In [8]:
# Use the scaler to transform the predicted values back to their original scale
predictions_testdata = loaded_scaler_originalval.inverse_transform(predictions)

print("Predicted Zoom, Predicted Focus, Predicted Contrast ")
print(predictions_testdata)

Predicted Zoom, Predicted Focus, Predicted Contrast 
[[2.1710621e+04 6.1790309e+00 6.8929832e+01]
 [4.3699918e+04 7.2661109e+00 6.8523415e+01]
 [2.1759262e+04 6.8399158e+00 6.7856842e+01]
 [4.9319180e+04 6.4465256e+00 6.9392853e+01]
 [2.2122727e+04 6.1949096e+00 6.8843987e+01]
 [4.5934176e+04 6.9459634e+00 6.7028366e+01]
 [8.1072568e+03 6.2194281e+00 6.8701950e+01]
 [1.0372207e+04 7.4517455e+00 6.8737587e+01]
 [8.2946680e+03 7.7040062e+00 6.7057861e+01]
 [2.7338906e+04 7.2092552e+00 6.7588333e+01]]


In [9]:
# Replace the relevant columns in the DataFrame with the predicted values
df_test[['Predicted Zoom', 'Predicted Focus', 'Predicted Contrast']] = predictions_testdata

# Save the DataFrame back to the same CSV file
df_test.to_csv("TestData.csv", index=False)